# New Dataset Loading and Cleansing

In [1]:
import pandas as pd
import numpy as np
import datetime
import timeit

In [2]:
df = pd.read_csv('new_data.csv', sep = ',')
df.head()

,_id,reservationNumber,scheduledTime,createdAt,startLoc,from,to,passengers,rider,serviceLevel,endLoc,acceptedTime,arrivedTime,rideStartedTime,rideEndTime
0,575fa95dab37c2772d83d133,2BB-EF32C,2016-06-14T06:50:00.000Z,2016-06-14T06:51:09.737Z,33.9315145;-84.2679194,"4633 Peachtree Place Parkway, Atlanta, GA, Uni...",NaN,1,573c17859534d60788663c28,5720cacf9b36887aef7fbaa6,33.9441649;-84.2415626,2016-06-14T06:51:18.260Z,2016-06-14T06:58:18.550Z,2016-06-14T06:58:21.177Z,2016-06-14T07:10:25.639Z
1,581ce7709a700718d82de733,RES-3M15GX,2016-11-04T19:53:00.000Z,2016-11-04T19:54:24.545Z,33.9528732;-84.233275,"3200 Holcomb Bridge Road, Norcross, GA, United...",NaN,1,572395516f7d187b4ab3d21a,5720cacf9b36887aef7fbaa6,33.9528732;-84.233275,NaN,2016-11-04T19:53:00.000Z,2016-11-04T19:53:00.000Z,2016-11-04T20:01:48.665Z
2,5760787cbd8c0b43b44dee43,2BE-0052C,2016-06-14T21:33:00.000Z,2016-06-14T21:34:53.116Z,33.9232324;-84.216982,"2250 Pelican Drive, Norcross, GA, United States",NaN,1,57220bef6095b2634daa4436,571f94066095b2634daa3f7a,33.9300262;-84.2212213,NaN,2016-06-14T21:44:20.512Z,2016-06-14T21:48:32.092Z,2016-06-14T22:07:46.735Z
3,577762cc10cbf2d3fe10d072,2F8-58F4B,2016-07-02T06:43:00.000Z,2016-07-02T06:44:29.092Z,33.8734443;-84.1894776,"510 Club Pkwy, Norcross, GA, United States",NaN,1,577762cca2d3758e115fabe5,571f94066095b2634daa3f7a,33.8734443;-84.1894776,NaN,2016-07-02T06:43:00.000Z,2016-07-02T06:43:00.000Z,2016-07-02T06:45:00.000Z
4,5824509d283a9767a95da152,RES-3ZKD1D,2016-11-10T10:48:00.000Z,2016-11-10T10:49:01.798Z,33.9467281;-84.2582733,"313 Ashley Lakes Drive, Norcross, GA, United S...",NaN,1,5732ff7d93731471b2459af8,5720cacf9b36887aef7fbaa6,33.9467281;-84.2582733,NaN,2016-11-10T10:48:00.000Z,2016-11-10T10:48:00.000Z,2016-11-10T11:12:07.421Z


In [3]:
# count number of nan in endLoc
df[df['endLoc'].isnull()].shape

(4310, 15)

In [4]:
# drop nan in endLoc
df = df.dropna(subset = ['endLoc'])
df.shape

(55685, 15)

In [5]:
# split the string in starting and ending location to create coordinates variables
df['start_lt'] = df['startLoc'].apply(lambda x: x.split(';')[0])
df['start_lg'] = df['startLoc'].apply(lambda x: x.split(';')[1])
df['end_lt'] = df['endLoc'].apply(lambda x: x.split(';')[0])
df['end_lg'] = df['endLoc'].apply(lambda x: x.split(';')[1])

In [6]:
df['serviceLevel'].replace(to_replace = '571f94066095b2634daa3f7a', value = 'Any vehicle', inplace = True)
df['serviceLevel'].replace(to_replace = '5720cacf9b36887aef7fbaa6', value = 'Wheelchair-accessible', inplace = True)
df['serviceLevel'].replace(to_replace = '5728a0b84b4be25e7aec2a8c', value = 'Any vehicle', inplace = True)
df['serviceLevel'].replace(to_replace = '572e01e12db0c8729f9859af', value = 'Van', inplace = True)
df['serviceLevel'].replace(to_replace = '58c7f9f39dc296a494091494', value = 'Any vehicle', inplace = True)

# filter the dataframe to only include service level = any vehicle
df = df[df['serviceLevel'] == 'Any vehicle']

In [7]:
df.head()

,_id,reservationNumber,scheduledTime,createdAt,startLoc,from,to,passengers,rider,serviceLevel,endLoc,acceptedTime,arrivedTime,rideStartedTime,rideEndTime,start_lt,start_lg,end_lt,end_lg
2,5760787cbd8c0b43b44dee43,2BE-0052C,2016-06-14T21:33:00.000Z,2016-06-14T21:34:53.116Z,33.9232324;-84.216982,"2250 Pelican Drive, Norcross, GA, United States",NaN,1,57220bef6095b2634daa4436,Any vehicle,33.9300262;-84.2212213,NaN,2016-06-14T21:44:20.512Z,2016-06-14T21:48:32.092Z,2016-06-14T22:07:46.735Z,33.9232324,-84.216982,33.9300262,-84.2212213
3,577762cc10cbf2d3fe10d072,2F8-58F4B,2016-07-02T06:43:00.000Z,2016-07-02T06:44:29.092Z,33.8734443;-84.1894776,"510 Club Pkwy, Norcross, GA, United States",NaN,1,577762cca2d3758e115fabe5,Any vehicle,33.8734443;-84.1894776,NaN,2016-07-02T06:43:00.000Z,2016-07-02T06:43:00.000Z,2016-07-02T06:45:00.000Z,33.8734443,-84.1894776,33.8734443,-84.1894776
8,575f5840d95097562af054c3,2BB-5965C,2016-06-14T01:03:00.000Z,2016-06-14T01:05:04.969Z,33.9470217;-84.238719,"Crossland Atlanta - Peachtree Corners, Jimmy C...",NaN,1,575f583fd95097562af054c0,Any vehicle,33.9344154;-84.1834709,NaN,2016-06-14T01:11:52.532Z,2016-06-14T01:17:58.987Z,2016-06-14T01:29:43.788Z,33.9470217,-84.238719,33.9344154,-84.1834709
9,575f467a74413a512acf9478,2BB-2775B,2016-06-13T23:48:00.000Z,2016-06-13T23:49:14.607Z,33.879998;-84.54384,"3007 Nursery Road Southeast, Smyrna, GA, Unite...",NaN,1,5745f470ec29abd620058930,Any vehicle,33.8754391;-84.5333324,2016-06-14T00:12:04.371Z,2016-06-14T00:19:12.638Z,2016-06-14T00:19:15.433Z,2016-06-14T00:23:51.420Z,33.879998,-84.54384,33.8754391,-84.5333324
10,577322d2633e8e38a42ed2f9,2EC-E473F,2016-06-29T01:20:00.000Z,2016-06-29T01:22:27.059Z,33.8257829;-84.4912726,"5230 S Cobb Dr, Smyrna, GA, United States",NaN,1,5772c5ece1306833a4192bf1,Any vehicle,33.8257829;-84.4912726,NaN,2016-06-29T01:20:00.000Z,2016-06-29T01:20:00.000Z,2016-06-29T02:04:20.692Z,33.8257829,-84.4912726,33.8257829,-84.4912726


In [8]:
# rename the '_id' column
df.rename(columns = {'_id':'id'}, inplace = True)

In [10]:
df.head()

,id,reservationNumber,scheduledTime,createdAt,passengers,rider,serviceLevel,rideStartedTime,rideEndTime,start_lt,start_lg,end_lt,end_lg
2,5760787cbd8c0b43b44dee43,2BE-0052C,2016-06-14T21:33:00.000Z,2016-06-14T21:34:53.116Z,1,57220bef6095b2634daa4436,Any vehicle,2016-06-14T21:48:32.092Z,2016-06-14T22:07:46.735Z,33.9232324,-84.216982,33.9300262,-84.2212213
3,577762cc10cbf2d3fe10d072,2F8-58F4B,2016-07-02T06:43:00.000Z,2016-07-02T06:44:29.092Z,1,577762cca2d3758e115fabe5,Any vehicle,2016-07-02T06:43:00.000Z,2016-07-02T06:45:00.000Z,33.8734443,-84.1894776,33.8734443,-84.1894776
8,575f5840d95097562af054c3,2BB-5965C,2016-06-14T01:03:00.000Z,2016-06-14T01:05:04.969Z,1,575f583fd95097562af054c0,Any vehicle,2016-06-14T01:17:58.987Z,2016-06-14T01:29:43.788Z,33.9470217,-84.238719,33.9344154,-84.1834709
9,575f467a74413a512acf9478,2BB-2775B,2016-06-13T23:48:00.000Z,2016-06-13T23:49:14.607Z,1,5745f470ec29abd620058930,Any vehicle,2016-06-14T00:19:15.433Z,2016-06-14T00:23:51.420Z,33.879998,-84.54384,33.8754391,-84.5333324
10,577322d2633e8e38a42ed2f9,2EC-E473F,2016-06-29T01:20:00.000Z,2016-06-29T01:22:27.059Z,1,5772c5ece1306833a4192bf1,Any vehicle,2016-06-29T01:20:00.000Z,2016-06-29T02:04:20.692Z,33.8257829,-84.4912726,33.8257829,-84.4912726


In [11]:
# transform the date time columns data type from string to datetime
df['scheduledTime'] = pd.to_datetime(df['scheduledTime'][:-5],format = '%Y-%m-%dT%H:%M:%S.')
df['createdAt'] = pd.to_datetime(df['createdAt'][:-5],format = '%Y-%m-%dT%H:%M:%S.')
df['rideStartedTime'] = pd.to_datetime(df['rideStartedTime'][:-5],format = '%Y-%m-%dT%H:%M:%S.')
df['rideEndTime'] = pd.to_datetime(df['rideEndTime'][:-5],format = '%Y-%m-%dT%H:%M:%S.')
df.head()

,id,reservationNumber,scheduledTime,createdAt,passengers,rider,serviceLevel,rideStartedTime,rideEndTime,start_lt,start_lg,end_lt,end_lg
2,5760787cbd8c0b43b44dee43,2BE-0052C,2016-06-14 21:33:00,2016-06-14 21:34:53.116,1,57220bef6095b2634daa4436,Any vehicle,2016-06-14 21:48:32.092,2016-06-14 22:07:46.735,33.9232324,-84.216982,33.9300262,-84.2212213
3,577762cc10cbf2d3fe10d072,2F8-58F4B,2016-07-02 06:43:00,2016-07-02 06:44:29.092,1,577762cca2d3758e115fabe5,Any vehicle,2016-07-02 06:43:00.000,2016-07-02 06:45:00.000,33.8734443,-84.1894776,33.8734443,-84.1894776
8,575f5840d95097562af054c3,2BB-5965C,2016-06-14 01:03:00,2016-06-14 01:05:04.969,1,575f583fd95097562af054c0,Any vehicle,2016-06-14 01:17:58.987,2016-06-14 01:29:43.788,33.9470217,-84.238719,33.9344154,-84.1834709
9,575f467a74413a512acf9478,2BB-2775B,2016-06-13 23:48:00,2016-06-13 23:49:14.607,1,5745f470ec29abd620058930,Any vehicle,2016-06-14 00:19:15.433,2016-06-14 00:23:51.420,33.879998,-84.54384,33.8754391,-84.5333324
10,577322d2633e8e38a42ed2f9,2EC-E473F,2016-06-29 01:20:00,2016-06-29 01:22:27.059,1,5772c5ece1306833a4192bf1,Any vehicle,2016-06-29 01:20:00.000,2016-06-29 02:04:20.692,33.8257829,-84.4912726,33.8257829,-84.4912726


In [13]:
df['reservationTime'] = df[['scheduledTime', 'createdAt']].min(axis = 1)
df.head()

,id,reservationNumber,scheduledTime,createdAt,passengers,rider,serviceLevel,rideStartedTime,rideEndTime,start_lt,start_lg,end_lt,end_lg,reservationTime
2,5760787cbd8c0b43b44dee43,2BE-0052C,2016-06-14 21:33:00,2016-06-14 21:34:53.116,1,57220bef6095b2634daa4436,Any vehicle,2016-06-14 21:48:32.092,2016-06-14 22:07:46.735,33.9232324,-84.216982,33.9300262,-84.2212213,2016-06-14 21:33:00
3,577762cc10cbf2d3fe10d072,2F8-58F4B,2016-07-02 06:43:00,2016-07-02 06:44:29.092,1,577762cca2d3758e115fabe5,Any vehicle,2016-07-02 06:43:00.000,2016-07-02 06:45:00.000,33.8734443,-84.1894776,33.8734443,-84.1894776,2016-07-02 06:43:00
8,575f5840d95097562af054c3,2BB-5965C,2016-06-14 01:03:00,2016-06-14 01:05:04.969,1,575f583fd95097562af054c0,Any vehicle,2016-06-14 01:17:58.987,2016-06-14 01:29:43.788,33.9470217,-84.238719,33.9344154,-84.1834709,2016-06-14 01:03:00
9,575f467a74413a512acf9478,2BB-2775B,2016-06-13 23:48:00,2016-06-13 23:49:14.607,1,5745f470ec29abd620058930,Any vehicle,2016-06-14 00:19:15.433,2016-06-14 00:23:51.420,33.879998,-84.54384,33.8754391,-84.5333324,2016-06-13 23:48:00
10,577322d2633e8e38a42ed2f9,2EC-E473F,2016-06-29 01:20:00,2016-06-29 01:22:27.059,1,5772c5ece1306833a4192bf1,Any vehicle,2016-06-29 01:20:00.000,2016-06-29 02:04:20.692,33.8257829,-84.4912726,33.8257829,-84.4912726,2016-06-29 01:20:00


In [16]:
# keep only the relevant columns
col_keep = ['id', 'reservationNumber', 'passengers', 'rider', 'serviceLevel', 'reservationTime', 'rideStartedTime', 'rideEndTime', 'start_lt', 'start_lg', 'end_lt', 'end_lg']
df = df[col_keep]
df.head()

,id,reservationNumber,passengers,rider,serviceLevel,reservationTime,rideStartedTime,rideEndTime,start_lt,start_lg,end_lt,end_lg
2,5760787cbd8c0b43b44dee43,2BE-0052C,1,57220bef6095b2634daa4436,Any vehicle,2016-06-14 21:33:00,2016-06-14 21:48:32.092,2016-06-14 22:07:46.735,33.9232324,-84.216982,33.9300262,-84.2212213
3,577762cc10cbf2d3fe10d072,2F8-58F4B,1,577762cca2d3758e115fabe5,Any vehicle,2016-07-02 06:43:00,2016-07-02 06:43:00.000,2016-07-02 06:45:00.000,33.8734443,-84.1894776,33.8734443,-84.1894776
8,575f5840d95097562af054c3,2BB-5965C,1,575f583fd95097562af054c0,Any vehicle,2016-06-14 01:03:00,2016-06-14 01:17:58.987,2016-06-14 01:29:43.788,33.9470217,-84.238719,33.9344154,-84.1834709
9,575f467a74413a512acf9478,2BB-2775B,1,5745f470ec29abd620058930,Any vehicle,2016-06-13 23:48:00,2016-06-14 00:19:15.433,2016-06-14 00:23:51.420,33.879998,-84.54384,33.8754391,-84.5333324
10,577322d2633e8e38a42ed2f9,2EC-E473F,1,5772c5ece1306833a4192bf1,Any vehicle,2016-06-29 01:20:00,2016-06-29 01:20:00.000,2016-06-29 02:04:20.692,33.8257829,-84.4912726,33.8257829,-84.4912726


In [17]:
df.to_csv('new_dataframe.csv', sep = ',', index = False)